In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
from torch.optim import AdamW

In [3]:


# Define dataset
class YouTubeCommentDataset(Dataset):
    def __init__(self, texts, spam_labels, sentiment_labels, toxicity_labels, hate_speech_labels, tokenizer, max_len=128):
        self.texts = texts
        self.spam_labels = spam_labels
        self.sentiment_labels = sentiment_labels
        self.toxicity_labels = toxicity_labels
        self.hate_speech_labels = hate_speech_labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'spam_label': torch.tensor(self.spam_labels[idx], dtype=torch.float),
            'sentiment_label': torch.tensor(self.sentiment_labels[idx], dtype=torch.long),
            'toxicity_labels': torch.tensor(self.toxicity_labels[idx], dtype=torch.float),
            'hate_speech_label': torch.tensor(self.hate_speech_labels[idx], dtype=torch.long),
        }

# Define model
class MultiTaskXLMRoberta(nn.Module):
    def __init__(self, model_name, num_sentiment_classes, num_toxicity_labels, num_hate_speech_classes):
        super(MultiTaskXLMRoberta, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.spam_head = nn.Linear(hidden_size, 1)
        self.sentiment_head = nn.Linear(hidden_size, num_sentiment_classes)
        self.toxicity_head = nn.Linear(hidden_size, num_toxicity_labels)
        self.hate_speech_head = nn.Linear(hidden_size, num_hate_speech_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]  # XLM-Roberta CLS token
        cls_output = self.dropout(cls_output)
        
        return {
            'spam': self.spam_head(cls_output),
            'sentiment': self.sentiment_head(cls_output),
            'toxicity': self.toxicity_head(cls_output),
            'hate_speech': self.hate_speech_head(cls_output)
        }

# Training function
def train(model, train_loader, val_loader, device, epochs=5):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 0, total_steps)

    spam_loss_fn = nn.BCEWithLogitsLoss()
    sentiment_loss_fn = nn.CrossEntropyLoss()
    toxicity_loss_fn = nn.BCEWithLogitsLoss()
    hate_loss_fn = nn.CrossEntropyLoss()

    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            spam_label = batch['spam_label'].to(device)
            sentiment_label = batch['sentiment_label'].to(device)
            toxicity_labels = batch['toxicity_labels'].to(device)
            hate_label = batch['hate_speech_label'].to(device)

            outputs = model(input_ids, attention_mask)

            loss = (
                spam_loss_fn(outputs['spam'].squeeze(), spam_label) +
                sentiment_loss_fn(outputs['sentiment'], sentiment_label) +
                toxicity_loss_fn(outputs['toxicity'], toxicity_labels) +
                hate_loss_fn(outputs['hate_speech'], hate_label)
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}")

# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()

    all_spam_preds, all_spam_labels = [], []
    all_sentiment_preds, all_sentiment_labels = [], []
    all_toxicity_preds, all_toxicity_labels = [], []
    all_hate_preds, all_hate_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            spam_label = batch['spam_label'].to(device)
            sentiment_label = batch['sentiment_label'].to(device)
            toxicity_labels = batch['toxicity_labels'].to(device)
            hate_label = batch['hate_speech_label'].to(device)

            outputs = model(input_ids, attention_mask)

            all_spam_preds.extend(torch.sigmoid(outputs['spam']).squeeze().cpu().numpy() > 0.5)
            all_spam_labels.extend(spam_label.cpu().numpy())

            all_sentiment_preds.extend(torch.argmax(outputs['sentiment'], dim=1).cpu().numpy())
            all_sentiment_labels.extend(sentiment_label.cpu().numpy())

            all_toxicity_preds.extend((torch.sigmoid(outputs['toxicity']).cpu().numpy() > 0.5).astype(int))
            all_toxicity_labels.extend(toxicity_labels.cpu().numpy())

            all_hate_preds.extend(torch.argmax(outputs['hate_speech'], dim=1).cpu().numpy())
            all_hate_labels.extend(hate_label.cpu().numpy())

    # Metrics
    print("\nSpam Classification Report")
    print(f"Accuracy: {accuracy_score(all_spam_labels, all_spam_preds):.4f}")
    print(f"F1 Score: {f1_score(all_spam_labels, all_spam_preds):.4f}")

    print("\nSentiment Classification Report")
    print(f"Accuracy: {accuracy_score(all_sentiment_labels, all_sentiment_preds):.4f}")
    print(f"F1 Macro: {f1_score(all_sentiment_labels, all_sentiment_preds, average='macro'):.4f}")

    print("\nToxicity Classification Report (Multi-Label)")
    print(f"F1 Macro: {f1_score(np.array(all_toxicity_labels), np.array(all_toxicity_preds), average='macro'):.4f}")

    print("\nHate Speech Classification Report")
    print(f"Accuracy: {accuracy_score(all_hate_labels, all_hate_preds):.4f}")
    print(f"F1 Macro: {f1_score(all_hate_labels, all_hate_preds, average='macro'):.4f}")

# Main
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load and clean data
    df = pd.read_csv("../datasets/processed/final_train_data.csv")

    # Tasks info
    tasks = {
        'spam': 'spam',
        'sentiment': 'sentiment',
        'toxicity': ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
        'hate_speech': 'hate_speech'
    }

    # Clean columns
    df['spam'] = pd.to_numeric(df['spam'], errors='coerce').fillna(0).astype(int)
    df['hate_speech'] = pd.to_numeric(df['hate_speech'], errors='coerce').fillna(0).astype(int)
    for col in tasks['toxicity']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Map sentiment text to indices
    sentiment_map = {'Positive': 0, 'Neutral': 1, 'Negative': 2, 'Irrelevant': 3}
    df['sentiment'] = df['sentiment'].map(sentiment_map).fillna(1).astype(int)

    # Prepare data
    X = df['text'].fillna("").tolist()
    spam_y = df['spam'].tolist()
    sentiment_y = df['sentiment'].tolist()
    toxicity_y = df[tasks['toxicity']].values.tolist()
    hate_y = df['hate_speech'].tolist()

    # Split
    train_idx, val_idx = train_test_split(range(len(X)), test_size=0.2, random_state=42)
    
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

    train_dataset = YouTubeCommentDataset(
        [X[i] for i in train_idx],
        [spam_y[i] for i in train_idx],
        [sentiment_y[i] for i in train_idx],
        [toxicity_y[i] for i in train_idx],
        [hate_y[i] for i in train_idx],
        tokenizer
    )

    val_dataset = YouTubeCommentDataset(
        [X[i] for i in val_idx],
        [spam_y[i] for i in val_idx],
        [sentiment_y[i] for i in val_idx],
        [toxicity_y[i] for i in val_idx],
        [hate_y[i] for i in val_idx],
        tokenizer
    )

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)

    # Model
    model = MultiTaskXLMRoberta(
        model_name="xlm-roberta-base",
        num_sentiment_classes=4,
        num_toxicity_labels=6,
        num_hate_speech_classes=3
    )

    # Train
    train(model, train_loader, val_loader, device, epochs=5)

    # Evaluate
    evaluate(model, val_loader, device)

if __name__ == "__main__":
    main()


Using device: cuda


C:\Users\Q\AppData\Local\Temp\ipykernel_39688\2529523849.py:156: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datasets/processed/final_train_data.csv")
Training Epoch 1: 100%|██████████| 9856/9856 [1:03:18<00:00,  2.59it/s]   


Epoch 1 - Loss: 0.4164


Training Epoch 2: 100%|██████████| 9856/9856 [1:51:42<00:00,  1.47it/s]     


Epoch 2 - Loss: 0.2631


...

Epoch 4 - Loss: 0.1001


Training Epoch 5: 100%|██████████| 9856/9856 [1:09:39<00:00,  2.36it/s]  


Epoch 5 - Loss: 0.0656


Evaluating: 100%|██████████| 1232/1232 [03:09<00:00,  6.49it/s]



Spam Classification Report
Accuracy: 0.9989
F1 Score: 0.8750

Sentiment Classification Report
Accuracy: 0.9642
F1 Macro: 0.9093

Toxicity Classification Report (Multi-Label)
F1 Macro: 0.5466

Hate Speech Classification Report
Accuracy: 1.0000
F1 Macro: 1.0000


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3060 Laptop GPU
